In [ ]:
# Compute Capex

## Import requires libraries and APIs

import glob
import io
import os
import time
import zipfile
from dateutil import rrule
from datetime import datetime, timedelta, date
import requests
import numpy as np
import pandas as pd

from google.colab import auth 
auth.authenticate_user()

## Mount google drive
from google.colab import drive
drive.mount("/content/gdrive")

from scipy.optimize import linprog

root_path = 'gdrive/My Drive/Consulting/2022 ZeroAvia (Maximov, Consulting)/2022-12 Davos Forum/Datasets/'

Mounted at /content/gdrive


In [ ]:
economics = {
    "irr" : 0.12, #internal rate of return
    "lifetime" : 20, 
    "inflation" : 0.0292, # mean over years
    "renew_time": 10, 
    "tax_shield": 0.3, # IRA
    "fte_cost" : 250000, # annual slary with taxes
    "min_fte" : 3, # minimal # of people
    "contingency": 0.10, # x = real price, take 1.1 x 
    "n_days": 365, 
    "n_hours": 24,
    "lcfs": -0.79, # LCFS discount applicable state-wide to California
    "tax_rate": 0.21 #corporate tax rate
}

battery = {
    "change_time" : 10, # to change on "lifetime"
    "annual_degradation" : 0.02,
    "depth_of_charge" : 0.9, # 90%
    "annual_cost_drop" : 0.03, # FY 22: x rub => FY 23 0.97 x rub => FY 24 : (0.97)^2 x
    "lifetime" : 10, 
    "second_life_market_value" : 0.53, # sell in 10 years for 0.53 x
    "tax_shield" : 0.3,
    "degradation": 0.02, 
    "loss": 0.02
}

solar = {
    "m2_in_kw" : 4.8, # to be removed
    "power_to_energy_factor" : 0.6, # C factor
    "second_life_market_value" : 0.36, # sell in Y years for 0.36 x
    "annual_cost_drop" : 0.025, 
    "tax_shield" : 0.3,
    "degradation": 0.005, # 0.5% per year
    "loss": 0.005
}

electrolyzer = {
    "FY22_efficiency" : 0.6, # heating value 54, 1 kg H2 = 33.4 kWh
    "annual_efficiency_increase" : 0.015, 
    "maximal_efficiency" : 0.85,
    "annual_cost_drop" : 0.1,
    "tax_shield" : 0.3,
    "lifetime_years" : 10,
    "degradation": 0.02 # up to 2% for C = 0.3
}

grid = {
    "loss": 0.005
}

In [17]:

#compute_GH2_price("SJC", "b20", "green") #9.162018925482965
#compute_GH2_price("SFO", "wdat", "green") #8.467853531122557
#print("===============================")
#compute_GH2_price("SJC", "b20", "brown") #
compute_GH2_price("SJC", "wdat", "green")


#print("===============================")
#compute_GH2_price("MRY", "b20", "green")
#compute_GH2_price("MRY", "wdat", "green")
#compute_GH2_price("MRY", "b20", "brown")

#print("===============================")
#compute_GH2_price("MRY", "wdat", "brown")
#compute_GH2_price("MRY", "wdat", "green")


h2_demand 25997
electrolyzer default:  79.81364986749016
solar power, cost 383.10551936395274 271295101.6173972
opex 3058021.1435191934
capex 154999158.36194432
lcoe 44.30831702773836
h2_opt, x_opt, hours_opt 0.0 [0. 0. 0. 0.] 10.34
optimal confiiguration: battery, MWh  0.0 electrolyzer, MW  0.0 solar, MW  0.0
solar power, cost 0.0 1205861.554282778
opex 0.0
capex 48300.0
lcoe inf
unit energy cost 0.0
COST (h2 opr), opt_hours, total system cost, price plan 0.0 10.34 nan wdat
check up -- H2 cost nan


<ipython-input-11-bc70ed98355c>:28: RuntimeWarning: divide by zero encountered in double_scalars
  lcoe = solar_cost/solar_power
<ipython-input-16-a21acf917c97>:40: RuntimeWarning: divide by zero encountered in double_scalars
  cost *= np.power(1.011,20/electrolyzer_power)
<ipython-input-16-a21acf917c97>:40: RuntimeWarning: invalid value encountered in double_scalars
  cost *= np.power(1.011,20/electrolyzer_power)


In [ ]:
def compute_GH2_price(airport, pricing_plan, h2_color):
  
  ##############################################################################
  ## Read the data from a file containing the grid price and the airport demand
  ## 
  ## airport.csv -- contains "ORIGIN_AIRPORT", "GRID_COST", "SOLAR_HOURS" and 
  ##                "KG_H2_DAY" columns
  ##
  ##############################################################################
  
  
  df = pd.read_csv(root_path+'airport.csv', index_col = None, header=0, low_memory=False)
  
  # filter by the airport we need
  df = df[df["ORIGIN_AIRPORT"] == airport]
  #print("df", df)

  # get solar hours
  solar_hours = df['SOLAR_HOURS'].values[0]
  #print("solar_hours", solar_hours)

  # get transmission grid cost =>> Change to demand_charge and energy_charge
  transmission_cost = df['TRANSMISSION_COST'].values[0]
  #print("transmission_cost", transmission_cost)

  # get overall demand
  h2_demand = df['KG_H2_DAY'].values[0]
  print("h2_demand", h2_demand)
  
  ############################################################################
  ## Correction on the system degradation: 
  ## 
  ## Use maximal degradation among -- electrolyzer(2%), battery (2%), and
  ##                                  solar (0.5%)
  ##
  ############################################################################
  

  # system degradation stands for the maximal degradation over the components
  degradation_rate = 0.02 ## =>> change on the component wise
  degradation_correction = np.power(1-degradation_rate, economics["renew_time"]+1)

  #print("degradation correction", degradation_correction)
  #print("h2 demand pre correction", h2_demand)

  # correct hydrogen request
  h2_demand *=  (1/degradation_correction)
  #print("h2 demand after correction", h2_demand)
  
  
  # discounted volume of hydrogen production 
  discounted_volume = economics["n_days"]*h2_demand
  discounted_volume *= (1 - np.power(1/(1+economics["irr"]), economics["renew_time"]+1))/(1 - 1/(1+economics["irr"]))
  discounted_volume *= (economics["lifetime"]/economics["renew_time"])

  # discounted volume
  #print("discounted_volume", discounted_volume)

  ############################################################################
  ## setup a default optimal configuration that later on will be improved
  ## The defaults are based on the a precomputed config with 10 tonn demsn
  ############################################################################

  # default demand to elсectrolyzer power standing for 24 hours of operations
  d2e_rate = 0.059 
  
  #adjestment for the market value and 2025 technology
  #market_adjustment = 1.27

  # print 
  print("electrolyzer default: ", h2_demand*d2e_rate/24)
  
  electrolyzer_power = h2_demand*d2e_rate/24
  solar_power = 4.8*electrolyzer_power #20*24/5
  battery_capacity = 24*electrolyzer_power

  #print("electrolyzer_power, solar_power, battery_capacity", electrolyzer_power, solar_power, battery_capacity)
  # system_degradation = 0.015
  # rate = (1- system_degradation)/(1+economics["irr"])
  
  #energy costs, we use average than correct them => to make the code real time
  demand_charge = energy_charge = 0
  if (pricing_plan == "b20"):
    demand_charge = 108.23
    energy_charge = 202.0
  else: 
    demand_charge = 0
    energy_charge = df['TRANSMISSION_COST'].values[0]

  ############################################################################
  ###### Plan and color specific optimization 
  ############################################################################

  
  
  # minimal number of electrolyzer hours
  # h = 6

  # levelized cost per unit of connection that is equal in power to electrolyzer
  uc = get_connection_cost(pricing_plan, electrolyzer_power)/(discounted_volume*electrolyzer_power)
  # levelized cost per unit of electrolyzer
  ue = get_electrolyzer_cost(electrolyzer_power, h2_demand)/(discounted_volume*electrolyzer_power)
  # levelized cost of solar per MW
  us = get_solar_cost(solar_power)/(discounted_volume*solar_power)
  #print("unit cost solar per MW", get_solar_cost(solar_power)/solar_power)
  # levelized cost of battery per MWh
  ub = get_battery_cost(battery_capacity)/(discounted_volume*battery_capacity)
  # levelized cost of grid per MWh of daily demand

  # labor cost. The labor cost is inflated with the IRS rate in time and discounted with the IRR
  ul = get_labor_cost(solar_power, electrolyzer_power)/discounted_volume

  #print("uc, ue, us, ub, ul", uc, ue, us, ub, ul)

  # optimal configuration
  # setup h2_opt sufficienly high so that the true optimum is lower
  h2_opt = 500
  # optimal argument, config
  x_opt = 0
  # the optimal number of hours of electrolyzer work
  hours_opt = solar_hours


  # The optimization program is non-convex and not acceptable by solvers;
  # however, we can add a simple linear search 
  n = 20

  # maximum allowed operating hours
  # taking less than 24 as the trading system shows 
  # adjustment is made to capture green vs brown, numerical simuations based
  h_max = 21 if ((h2_color == "green") & (pricing_plan != "b20")) else 24

  # temporal variables 
  #res_x = 0
  #res_fun = 0
  
  # search space for electrolyser load
  h = np.linspace(2*solar_hours,h_max,n)

  # grid usage permission
  g_bounds = (0, 0)

  

  # run a cycle over the working hours of electrolyzer
  # withing the cycle we find approximately optimal configuration and then 
  # compute its real cost
  for i in range(n):
      
      # energy_cost based on the number of hours adjustment
      ug = (energy_charge+demand_charge/h[i])*economics["n_days"]

      #print((energy_charge+demand_charge/h[i]), "(energy_charge+demand_charge/h[i])")
      #print(economics["n_days"], "ndays")

      # assume grid cost to be constant over time which reflects the country average
      # discount in time 
      ug *= ((1 - np.power(1/(1+economics["irr"]), economics["lifetime"]+1))/(1-1/(1+economics["irr"])))

      # the same for magnitude as the rest u* variabels
      ug *= (1/discounted_volume)
      #print("ug", ug)
      
      # evening comission satding for the price ratio, simulation based
      #transmission_adjustment = 0.8
      #ug = ug if (pricing_plan == "b20") else (1+transmission_adjustment)*ug

      # cost factor 
      # the assumption is that the grid is a big battery
      cost = [ub, ue+uc, us, ug]
      #print("cost", cost)
      
      # linear programming proxy
      # based on the energy balance
      # variables "battery -- electrolyzer -- solar -- grid"
      #A = [[-(1-battery["loss"]), h[i], -(1-solar["loss"])*solar_hours, -(1-grid["loss"])], ...# energy balance el * h[i] - pv * (1-loss)*h_pv - (1-loss)*g - (1-loss)*b = 0
      #     [(1-battery["loss"]), -h[i], (1-solar["loss"])*solar_hours, (1-grid["loss"])] # make the above equality
      #     [1, -2, 0, 0]] # battery should be sufficient for at least 2 hours of electrolyzer power to count on contingency

      A = [[1, solar_hours, -solar_hours, 0], # energy balance: what is produced during the day goes to battery
           [-1, -solar_hours, solar_hours, 0], # to make the last equation
           [(1 - battery["loss"]), -(h[i] - solar_hours), 0 , -1],  # after the battery it may go to the grid or back to the electrolyser
           [-(1 - battery["loss"]), (h[i] - solar_hours), 0, 1], #,  # to make the last equation
           [-1, +2, 0, 0]] # short term safety constraint 

      b = [0, 0, 0, 0, 0]
      
      # Contstraints
      # battery is always non-negative
      b_bounds = (0, None)
      
      # electrolyzer works exactly the prescribed number of hours
      # we make a linear grid search on it
      e_bounds = (d2e_rate*h2_demand/h[i], d2e_rate*h2_demand/h[i])
      
      #print("1")
      # setup a reasonable maximal bound on the solar 
      s_bounds = (0, d2e_rate*h2_demand/2)

      
      # now setup plan and color specific bound on the grid
      # the assumption is that we can not use grid as a big battery
      g_bounds = (0, 0)
      
      #print("2")
      # Distribution + green
      if ((pricing_plan == "b20") & (h2_color == "green")):
        # we can not take something from the grid; an isolated microgrid
        g_bounds = (0, 0)
      elif ((pricing_plan == "b20") & (h2_color == "brown")):
        # we may sell energy to the grid for credits
        # lower bound -- the amount of energy taken from the grid (max)
        # upper bound -- the amount of energy given to the gris
        g_bounds = (0, None)
      elif (h2_color == "green"):
        # we can sell energy to the grid
        # 4/3 is the elasticity of demand by price
        # 5 MW is the typical size
        g_bounds = (0.75*5*(24-solar_hours), 0)
      else:
        #((pricing_plan != "b20") & (h2_color == "brown")):
        g_bounds = (0.75*5*(24-solar_hours), None)
      
      res = linprog(cost, A_ub=A, b_ub=b, bounds=[b_bounds, e_bounds, s_bounds, g_bounds])
      #print("res", res)
      if (res.fun < h2_opt):
        #print("current optimum", h[i])
        h2_opt = (res.fun)
        x_opt = res.x
        hours_opt = h[i]


  print("h2_opt, x_opt, hours_opt", h2_opt, x_opt, hours_opt)

  # fraction of indirect fees applicable to energy trading
  # comission = 0.3
  # market adjustment for simplification
  #model_mismatch = 1.0
  #if ((pricing_plan == "b20") & (h2_demand < 10000)):
  #  model_mismatch = - 0.07*(10000 - h2_demand)/10000 + 1
  #elif ((pricing_plan == "b20") & (h2_demand < 60000)):
  #  model_mismatch = 1.57 + 0.57*(h2_demand - 60000)/50000
  #elif (pricing_plan == "b20"):
  #  model_mismatch = 1.57

  print("optimal confiiguration:", "battery, MWh ", np.round(x_opt[0],0), "electrolyzer, MW ", np.round(x_opt[1],0),  "solar, MW ", np.round(x_opt[2],0))
  #print("battery (capacity, total cost, unit cost):", np.round(x_opt[0],0), get_battery_cost(np.round(x_opt[0],0)), get_battery_cost(np.round(x_opt[0],0))/np.round(x_opt[0],0))
  #print("electrolyser (power, total cost, unit cost):", np.round(x_opt[1],0), get_electrolyzer_cost(np.round(x_opt[1],0), h2_demand), get_electrolyzer_cost(np.round(x_opt[1],0), h2_demand)/np.round(x_opt[1],0))
  #print("solar (power, total cost, unit cost):", np.round(x_opt[2],0), get_solar_cost(np.round(x_opt[2],0)), get_solar_cost(np.round(x_opt[2],0))/np.round(x_opt[2],0))
  #print("grid (power, total cost, unit cost):", np.round(x_opt[1],0), get_connection_cost(pricing_plan, np.round(x_opt[1],0)), get_connection_cost(pricing_plan, np.round(x_opt[1],0))/(np.round(x_opt[1],0)))


  connection_cost = get_connection_cost(pricing_plan, np.round(x_opt[1],0))
  labor_cost = get_labor_cost(np.round(x_opt[2],0), np.round(x_opt[1],0))
  battery_cost = get_battery_cost(np.round(x_opt[0],0))
  solar_cost = get_solar_cost(np.round(x_opt[2],0))
  electrolyser_cost = get_electrolyzer_cost(np.round(x_opt[1],0), h2_demand)
  energy_cost = np.round(x_opt[2],0)*365*8
  print("unit energy cost", energy_cost/discounted_volume)

  cost = connection_cost + labor_cost + battery_cost + solar_cost + electrolyser_cost
  print("COST (h2 opr), opt_hours, total system cost, price plan", h2_opt, hours_opt, cost, pricing_plan)
  print("check up -- H2 cost", cost/discounted_volume)#/market_adjustment

  # apply LCSF
  h2_opt += economics["lcfs"]

  electrolyser_share = electrolyser_cost/(cost - labor_cost)
  battery_share = battery_cost/(cost - labor_cost)
  solar_share = solar_cost/(cost-labor_cost)
  grid_share = 1 - battery_share - electrolyser_share - solar_share
  #print("cost share (battery, electrolyser, solar, grid", battery_share, electrolyser_share, solar_share, grid_share)

  #print("h2_demand, hours_opt", h2_demand, hours_opt)
  #print("electrolyzer max production per day (kg)", h2_demand*24/hours_opt)
  #print("electrolyzer operating hours", hours_opt)

In [ ]:
# computes the money coming from energy trading
# constants are simulation based 

def compute_energy_cost(energy_need, mean_price, comission):
  return ((mean_price - comission)*mean_price if (energy_need < 0) else (mean_price + comission)*mean_price)
  

# CapEx & OpEx

In [ ]:
def configuration_cost(pricing_plan, electrolyzer_power, solar_power, battery_capacity, h2_demand):
  cost = 0
  cost += get_connection_cost(pricing_plan, electrolyzer_power)
  cost += get_labor_cost(solar_power, electrolyzer_power)
  cost += get_battery_cost(battery_capacity)
  cost += get_solar_cost(solar_power)
  cost += get_electrolyzer_cost(electrolyzer_power, h2_demand)
  return cost

In [ ]:
# no energy cost is included
def get_connection_cost(pricing_plan, connection_power):
  df = pd.read_csv(root_path+'grid.csv', index_col=None, header=0, low_memory=False)
  capex_fixed = capex_variable = opex_fixed = opex_variable = 0

  # filter for the right pricing plan
  df = df[df['grid'] == pricing_plan]

  capex_fixed = df.loc[(df["is_capex"] == "capex") & (df["is_fixed"] == "fixed"), "cost"].sum()
  capex_variable = df.loc[(df["is_capex"] == "capex") & (df["is_fixed"] == "variable"), "cost"].sum()
  capex = capex_fixed + connection_power*capex_variable

  # apply a tax shield, a part of the opex may also be treated under the tax shield umbrella
  capex *= (1 - economics["tax_shield"])
  
  # securities is a part of capex, but the infrastructure credit is not applicable to them
  securities_fixed = df.loc[np.where((df["is_capex"] == "secirities") & (df["is_fixed"] == "fixed"))]["cost"].sum()
  securities_variable = df.loc[np.where((df["is_capex"] == "secirities") & (df["is_fixed"] == "variable"))]["cost"].sum()
  securities = securities_fixed + connection_power*securities_variable

  ## compute operational expenses
  
  # annual opex
  opex_fixed = df.loc[(df["is_capex"] == "opex") & (df["is_fixed"] == "fixed"), "cost"].sum()
  #df.loc[np.where((df["is_capex"] == "opex") & (df["is_fixed"] == "fixed"))]["cost"].sum()
  opex_variable = df.loc[(df["is_capex"] == "capex") & (df["is_fixed"] == "variable"), "cost"].sum()
  #[np.where((df["is_capex"] == "opex") & (df["is_fixed"] == "variable"))]["cost"].sum()
  opex = opex_fixed + opex_variable*connection_power

  #overall opex
  opex = opex*(1-np.power(1/(1+economics["irr"]), economics["lifetime"]+1))/(1 - 1/(1+economics["irr"]))

  # real cost of security deposits
  securities_lifetime_corrector = 1 - np.power(1/(1+economics["irr"]), economics["lifetime"])
  opex_lifetime_corrector = (1 - np.power(1/(1+economics["irr"]), economics["lifetime"]))/(1 - 1/(1+economics["irr"]))

  if (pricing_plan == "caiso"):
    securities += securities_lifetime_corrector*(100000 if (connection_power < 20) else 250000)
  elif (pricing_plan == "b20"): 
    # add the cost of transmission upgrade as needed
    capex += (0 if (connection_power < 20) else 7500000)
  elif (pricing_plan == "wdat"):
    securities += securities_lifetime_corrector*(100000 if (connection_power < 20) else 15000000)

  # the economy of scale -- potentially need multiple connections if too big
  connection_cost = securities + capex + opex

  if (pricing_plan == "b20"):
  # 110 MVA is a practical threshold, however scaling should be proportional to as a standard allowance
    connection_cost = connection_cost * max(connection_power/40.0, 1)
  else:
  # 750 MVA max is a practical threshold
    connection_cost = connection_cost * max(connection_power/110.0, 1)

  return connection_cost


In [ ]:
# under the assumption of a properly sized microgrid
def get_labor_cost(solar_power, electrolyzer_power):
  headcount = max(0.025*solar_power + 0.05*electrolyzer_power,3)
  annual_cost = headcount*economics["fte_cost"]

  # correct for the project timeline
  lifetime_correction = (1-np.power((1+economics["inflation"])/(1+economics["irr"]), economics["lifetime"]+1))/(1 - (1+economics["inflation"])/(1+economics["irr"]))
  total_cost = annual_cost*lifetime_correction
  return total_cost 

In [ ]:
def get_battery_cost(battery_capacity):
  df = pd.read_csv(root_path+'battery.csv', index_col = None, header=0, low_memory = False)

  capex_unit = df["Capex_unit"].sum()
  opex_unit_year = df["Opex_unit"].sum()

  ## economy of scale

  # cost savings as a function of scale, 120 MWh
  scale_economy = 1
  if (battery_capacity < 120): 
    scale_economy = (battery_capacity/120+((1-battery_capacity/120)*525/428))
  elif (battery_capacity < 240):
    scale_economy = ((battery_capacity-120)/120)*(369/428)+(240-battery_capacity)/120
  else:
    scale_economy = max(np.exp(5.97 - 0.00026*battery_capacity)/428, 1.1*215/428)
  
  # applied to both capex and opex
  capex  = capex_unit*scale_economy*battery_capacity

  ## assume the battery change once in 10 years, later on we fix it throught simulations with a "heavy technoeconomics"
  
  # get a cost of a new battery respecting the scale
  new_battery_cost = capex*np.power((1-battery["annual_cost_drop"])/(1+economics["irr"]), battery["lifetime"])
  
  # resell the old battery and add the new one to the capex
  capex += new_battery_cost

  capex_resale = new_battery_cost*battery["second_life_market_value"]

  # resell the battery at the end of the project
  # first resale
  capex_resale += np.power((1-battery["annual_cost_drop"])/(1+economics["irr"]), battery["lifetime"] + 1)
  capex_resale += np.power((1-battery["annual_cost_drop"])/(1+economics["irr"]), economics["lifetime"] + 1)
  
  # account for the investment tax credit
  capex *= (1-battery["tax_shield"])
  
  # account for old battery reselling
  capex -= capex_resale 

  #print("battery unit capex", capex/battery_capacity)

  # correction of scale and capacity
  opex = opex_unit_year*battery_capacity*scale_economy

  # correction for project lifetime
  opex *= (1 - np.power(1/(1+economics["irr"]), economics["lifetime"]+1))
  opex *= (1/(1 - 1/(1+economics["irr"])))

  #print("battery unit opex", opex/battery_capacity)

  # we assume that the ITC correction is applied to Opex as well
  # as soon as (a) labor removed from the scope (b) maintenence can be treated as an upgrade and fall under the ITC scope
  #if (with_tax_shield):
  #  opex *= (1-battery["tax_shield"])

  battery_cost = capex + opex
  return battery_cost

In [ ]:
def get_solar_cost(solar_power):
   df = pd.read_csv(root_path+'solar.csv', index_col=None, header=0, low_memory=False)

   # hardware only cost
   capex_fixed = df.loc[np.where((df["is_capex"] == "capex"))]["Fixed"].sum()
   capex_variable = df.loc[np.where((df["is_capex"] == "capex"))]["Variable"].sum()
   capex = capex_fixed+capex_variable*solar_power

   # annual costs
   opex_fixed = df.loc[np.where((df["is_capex"] == "opex"))]["Fixed"].sum()
   opex_variable = df.loc[np.where((df["is_capex"] == "opex"))]["Variable"].sum()
   opex = opex_fixed+opex_variable*solar_power

   # compute overall opex for the project lifetime
   opex *= (np.power((1-solar["annual_cost_drop"])/(1+economics["irr"]), economics["lifetime"]+1)/(1 - (1-solar["annual_cost_drop"])/(1+economics["irr"])))

   # installation part of the capex
   installation_fixed = df.loc[np.where((df["is_capex"] == "installation"))]["Fixed"].sum()
   installation_variable = df.loc[np.where((df["is_capex"] == "installation"))]["Variable"].sum()
   installation = installation_fixed+installation_variable*solar_power

   capex_resale = solar["second_life_market_value"]*np.power((1-solar["annual_cost_drop"])/(1+economics["irr"]), economics["lifetime"]+1)*capex
   
   capex *= (1-solar["tax_shield"])
   solar_cost = capex + installation + opex - capex_resale

   # compute LCOE
   lcoe = solar_cost/solar_power
   factor = (1 - np.power(1/(1 + economics["irr"]), economics["lifetime"]+1))/(1-1/(1 + economics["irr"]))
   lcoe *= (1/(365*5.17*factor)) # median solar hours in this area, lifetime

   # solar_cost
   print("solar power, cost", solar_power, solar_cost)
   print("opex", opex)
   print("capex", capex)
   
   print("lcoe", lcoe)

   return solar_cost


In [ ]:
def get_electrolyzer_cost(electrolyzer_power, hydrogen_demand):
  df = pd.read_csv(root_path+'electrolyzer.csv', index_col=None, header=0, low_memory=False)

  cost = 0 

  # hardware only cost
  capex = df.loc[(df["is_capex"] == "capex"), "Variable"].sum()
  #capex = df.loc[np.where((df["is_capex"] == "capex"))]["Variable"].sum()*electrolyzer_power
  
  # apply the tax shield
  #capex = capex*(1-electrolyzer["tax_shield"])
  #if (with_tax_shield == True):
  capex = capex*(1-electrolyzer["tax_shield"])

  capex *= electrolyzer_power

  #print("capex", capex)

  # installation part of the capex
  installation = df.loc[(df["is_capex"] == "installation", "Variable")].sum()
  installation *= electrolyzer_power

  #print("installation", installation)

  cost += (installation + capex)
  #print("capex+installation, el", cost)

  # electrolyzer change after 10 years
  # IRA Tax credit is not applicable in 10 years
  
  efficiency_parity = electrolyzer["FY22_efficiency"]/(electrolyzer["FY22_efficiency"] + electrolyzer["annual_efficiency_increase"]*electrolyzer["lifetime_years"])

  # 2nd purchase
  cost += ((capex+installation)*np.power((1-electrolyzer["annual_cost_drop"])/(1+economics["irr"]), electrolyzer["lifetime_years"])*efficiency_parity)

  # make scale adjustment
  if (electrolyzer_power > 20):
    cost *= np.power(0.989,electrolyzer_power/20)
  else:
    cost *= np.power(1.011,20/electrolyzer_power)

  # add opexes one by one, inflated under different rates
  opex = electrolyzer_power*df.loc[(df["is_capex"] == "opex"), "Variable"].sum()
  #print("opex", opex)
  opex *= ((1 - np.power(1/(1+economics["irr"]), economics["lifetime"]+1))/(1 - 1/(1+economics["irr"])))

  # utility bills
  lifetime_multiplier = ((1 - np.power((1+economics["inflation"])/(1+economics["irr"]), economics["lifetime"]+1))/(1 - (1+economics["inflation"])/(1+economics["irr"])))

  opex_kg = (hydrogen_demand*df.loc[(df["is_capex"] == "utilities"), "Per_KG"].sum())

  cost += opex
  #print("system opex, el", opex)
  cost += opex_kg
  return cost
